In [5]:
# scripts/fix_db.py
import argparse
import os
import shutil
import sqlite3
from datetime import datetime
from pathlib import Path

def backup(db_path: str) -> str:
    bak = db_path + ".bak"
    shutil.copy2(db_path, bak)
    print(f"[backup] Copia creada: {bak}")
    return bak

def ensure_events_schema(conn: sqlite3.Connection) -> None:
    cols = [r[1] for r in conn.execute("PRAGMA table_info(events)").fetchall()]
    if "campaign" not in cols:
        print("[schema] Añadiendo columna events.campaign ...")
        conn.execute("ALTER TABLE events ADD COLUMN campaign TEXT")

def ensure_email_map_schema(conn: sqlite3.Connection) -> None:
    conn.execute("""
        CREATE TABLE IF NOT EXISTS email_map (
            msg_id   TEXT PRIMARY KEY,
            email    TEXT NOT NULL,
            send_ts  TEXT,
            campaign TEXT
        )
    """)
    cols = [r[1] for r in conn.execute("PRAGMA table_info(email_map)").fetchall()]
    if "send_ts" not in cols:
        print("[schema] Añadiendo columna email_map.send_ts ...")
        conn.execute("ALTER TABLE email_map ADD COLUMN send_ts TEXT")
    if "campaign" not in cols:
        print("[schema] Añadiendo columna email_map.campaign ...")
        conn.execute("ALTER TABLE email_map ADD COLUMN campaign TEXT")

def normalize_ts(conn: sqlite3.Connection, table: str, col: str) -> int:
    # Sustituye 'T' por espacio; mantiene microsegundos si existen
    cur = conn.execute(f"UPDATE {table} SET {col} = REPLACE({col}, 'T', ' ') WHERE {col} LIKE '%T%'")
    print(f"[normalize] {table}.{col}: {cur.rowcount} filas actualizadas")
    return cur.rowcount

def backfill_campaign_from_map(conn_events: sqlite3.Connection, conn_map: sqlite3.Connection) -> int:
    # Copia campaign desde email_map a events si está NULL
    # (SQLite permite subquery en SET)
    cur = conn_events.execute("""
        UPDATE events
           SET campaign = (
               SELECT campaign FROM email_map WHERE email_map.msg_id = events.msg_id
           )
         WHERE campaign IS NULL
           AND EXISTS (SELECT 1 FROM email_map WHERE email_map.msg_id = events.msg_id)
    """)
    print(f"[backfill] events.campaign rellenado en {cur.rowcount} filas")
    return cur.rowcount

def insert_missing_send_events(conn_events: sqlite3.Connection, conn_map: sqlite3.Connection) -> int:
    # msg_ids con send ya existentes
    existing = {row[0] for row in conn_events.execute(
        "SELECT msg_id FROM events WHERE event_type='send'"
    ).fetchall()}

    rows = conn_map.execute("SELECT msg_id, send_ts, campaign FROM email_map").fetchall()
    to_insert = []
    for msg_id, send_ts, campaign in rows:
        if not send_ts:
            continue
        if msg_id in existing:
            continue
        # Asegura formato YYYY-MM-DD HH:MM:SS[.ffffff]
        ts = str(send_ts).replace("T", " ")
        to_insert.append((msg_id, "send", None, ts, campaign))

    if not to_insert:
        print("[send] No hay 'send' pendientes de insertar.")
        return 0

    conn_events.executemany("""
        INSERT INTO events (msg_id, event_type, client_ip, ts, campaign)
        VALUES (?, ?, ?, ?, ?)
    """, to_insert)
    print(f"[send] Insertados {len(to_insert)} eventos 'send'")
    return len(to_insert)

def print_counts(conn_events: sqlite3.Connection) -> None:
    total = conn_events.execute("SELECT COUNT(*) FROM events").fetchone()[0]
    print(f"[stats] events total: {total}")
    for et in ("send", "open", "click", "unsubscribe", "complaint"):
        c = conn_events.execute("SELECT COUNT(*) FROM events WHERE event_type=?", (et,)).fetchone()[0]
        print(f"[stats] {et:12s}: {c}")

def main():
    parser = argparse.ArgumentParser(description="Arregla email_events.db y email_map.db")
    parser.add_argument("--events", default="../email_marketing/data/email_events.db",
                        help="Ruta a email_events.db")
    parser.add_argument("--map", default="../email_marketing/data/email_map.db",
                        help="Ruta a email_map.db")
    args, _ = parser.parse_known_args()

    events_db = Path(args.events)
    map_db = Path(args.map)

    if not events_db.exists():
        raise SystemExit(f"No existe: {events_db}")
    if not map_db.exists():
        raise SystemExit(f"No existe: {map_db}")

    backup(str(events_db))
    backup(str(map_db))

    with sqlite3.connect(str(events_db)) as ce, sqlite3.connect(str(map_db)) as cm:
        # 1) Esquemas
        ensure_events_schema(ce)
        ensure_email_map_schema(cm)

        # 2) Normaliza timestamps con 'T'
        ce.execute("BEGIN")
        normalize_ts(ce, "events", "ts")
        ce.commit()

        cm.execute("BEGIN")
        normalize_ts(cm, "email_map", "send_ts")
        cm.commit()

        # 3) Backfill campaign
        ce.execute("BEGIN")
        backfill_campaign_from_map(ce, cm)
        ce.commit()

        # 4) Inserta 'send' que falten
        ce.execute("BEGIN")
        inserted = insert_missing_send_events(ce, cm)
        ce.commit()

        # 5) Stats
        print_counts(ce)

    print("\n[OK] Migración completada.")

if __name__ == "__main__":
    main()


[backup] Copia creada: ..\email_marketing\data\email_events.db.bak
[backup] Copia creada: ..\email_marketing\data\email_map.db.bak
[normalize] events.ts: 8 filas actualizadas
[normalize] email_map.send_ts: 0 filas actualizadas


OperationalError: no such table: email_map